In [1]:

!pip install kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download dineshpiyasamara/sentiment-analysis-dataset
!unzip sentiment-analysis-dataset.zip

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/dineshpiyasamara/sentiment-analysis-dataset
License(s): unknown
  0% 0.00/460k [00:00<?, ?B/s]
100% 460k/460k [00:00<00:00, 24.2MB/s]
Archive:  sentiment-analysis-dataset.zip
  inflating: sentiment_analysis.csv  


In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.metrics import accuracy_score
import pandas as pd
import torch.nn.functional as F
import re
import nltk
from nltk.corpus import stopwords
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [3]:

df = pd.read_csv('sentiment_analysis.csv')
df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [4]:
df.set_index('id',inplace=True)
df.head()

,label,tweet
id,,
1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
2,0,Finally a transparant silicon case ^^ Thanks t...
3,0,We love this! Would you go? #talk #makememorie...
4,0,I'm wired I know I'm George I was made that wa...
5,1,What amazing service! Apple won't even talk to...


In [5]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean_text(text):
    text = text.lower()
    text = re.sub(r'<[^>]*>', '', text)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

def lemmatize_text(text):
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_tokens = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags
    ]
    return ' '.join(lemmatized_tokens)

def preprocess_text(text):
    text = clean_text(text)
    text = remove_stopwords(text)
    text = lemmatize_text(text)
    return text


df['tweet'] = df['tweet'].apply(preprocess_text)
df.head()

,label,tweet
id,,
1,0,fingerprint pregnancy test android apps beauti...
2,0,finally transparant silicon case thanks uncle ...
3,0,love would go talk makememories unplug relax i...
4,0,im wire know im george make way iphone cute da...
5,1,amazing service apple wont even talk question ...


In [6]:
all_text = ' '.join(df['tweet'])
vocab = {word:index for index,word  in enumerate(set(all_text.split()))}
vocab['<pad>'] = len(vocab)
vocab['<unk>'] = len(vocab)
print(len(vocab))

17061


In [41]:
class FakeNewsDataset(Dataset):
    def __init__(self,data_list,vocab,pad_token='<pad>'):
      self.data = data_list
      self.vocab = vocab
      self.pad_token = pad_token

    def __len__(self):
      return len(self.data)

    def __getitem__(self,idx):
      label,text = self.data[idx]
      text_indices = [self.vocab.get(word,self.vocab['<unk>']) for word in text.split()]
      label = torch.tensor(label, dtype = torch.float32)
      return label, text_indices
    def collate_fn(self,batch):
      labels,texts = zip(*batch)
      texts = [torch.tensor(text,dtype=torch.long) for text in texts]
      texts = nn.utils.rnn.pad_sequence(texts,batch_first=True,padding_value = self.vocab[self.pad_token])
      labels = torch.tensor(labels,dtype=torch.float32)
      return texts.to(device),labels.to(device)

train_size = int(0.7*len(df))
val_size = int(0.2*len(df))
test_size = int(0.1*len(df))
train_df,val_df,test_df = random_split(df.values, [train_size,val_size,test_size])
train_df = df.iloc[train_df.indices]
val_df = df.iloc[val_df.indices]
test_df = df.iloc[test_df.indices]
train_data_list = list(zip(train_df['label'],train_df['tweet']))
val_data_list = list(zip(val_df['label'],val_df['tweet']))
test_data_list = list(zip(test_df['label'],test_df['tweet']))
train_dataset = FakeNewsDataset(train_data_list,vocab)
val_dataset = FakeNewsDataset(val_data_list,vocab)
test_dataset = FakeNewsDataset(test_data_list,vocab)
train_dataloader = DataLoader(train_dataset,batch_size=16,shuffle=True,collate_fn=train_dataset.collate_fn,drop_last=True)
val_dataloader = DataLoader(val_dataset,batch_size=16,shuffle=False,collate_fn=val_dataset.collate_fn,drop_last=True)
test_dataloader = DataLoader(test_dataset,batch_size=16,shuffle=False,collate_fn=test_dataset.collate_fn,drop_last=False)


In [50]:
import torch
import torch.nn as nn

class FakeGRU(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout_prob, pretrained_embeddings=None):
        super(FakeGRU, self).__init__()

        # Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # Bidirectional GRU
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True, bidirectional=True, dropout=dropout_prob)
        # Attention Layer
        self.attention = nn.Linear(hidden_dim * 2, 1)  # For bidirectional GRU
        # Fully Connected Layers
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        # Dropout
        self.dropout = nn.Dropout(dropout_prob)
        # Layer Normalization
        self.layernorm = nn.LayerNorm(hidden_dim * 2)
        # Weight Initialization
        self.init_weights()

    def init_weights(self):
        for name, param in self.named_parameters():
            if 'weight' in name and param.dim() > 1:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0)

    def forward(self, text):
        embed = self.embedding(text)
        gru_out, _ = self.gru(embed)
        attention_weights = torch.softmax(self.attention(gru_out), dim=1)
        context_vector = torch.sum(attention_weights * gru_out, dim=1)
        context_vector = self.layernorm(context_vector)
        context_vector = self.dropout(context_vector)
        hidden = self.fc1(context_vector)
        hidden = self.dropout(hidden)
        return self.fc2(hidden)

vocab_size = len(vocab)
embedding_dim = 20
hidden_dim = 16
output_dim = 1
dropout_prob = 0.5

model = FakeGRU(vocab_size, embedding_dim, hidden_dim, output_dim, dropout_prob).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)


def train_model(model,dataloader,optimizer,criterion,max_grad_norm=1.0):
  model.train()
  total_loss = 0
  for texts, labels in dataloader:
    optimizer.zero_grad()
    outputs = model(texts).squeeze(1)
    loss = criterion(outputs,labels)
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(),max_grad_norm)
    optimizer.step()
    total_loss += loss.item()
  return total_loss/len(dataloader)

def evaluate_model(model,dataloader,criterion):
  model.eval()
  total_loss = 0
  all_preds = []
  all_labels = []
  with torch.no_grad():
    for texts,labels in dataloader:
      outputs = model(texts).squeeze(1)
      loss = criterion(outputs,labels)
      total_loss += loss.item()
      preds = torch.round(torch.sigmoid(outputs)).cpu().tolist()
      all_labels += labels.cpu().tolist()
      all_preds += preds
  accuracy = accuracy_score(all_labels,all_preds)
  f1 = f1_score(all_labels,all_preds)
  return total_loss/len(dataloader),accuracy,f1

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [51]:
best_val_loss = float('inf')
early_stop_counter = 0
patience = 5
num_epochs = 10

for epoch in range(num_epochs):
    train_loss = train_model(model, train_dataloader, optimizer, criterion)
    val_metrics = evaluate_model(model, val_dataloader, criterion)
    val_loss, val_accuracy, val_f1 = val_metrics

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, F1: {val_f1:.4f}")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")
        print("Best model saved!")
        early_stop_counter = 0

    else:
        optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] * 0.5
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered!")
            break

    scheduler.step(val_loss)



Epoch 1/10
Train Loss: 0.5046, Val Loss: 0.3519, Val Accuracy: 0.8491, F1: 0.7579
Best model saved!
Epoch 2/10
Train Loss: 0.2597, Val Loss: 0.3400, Val Accuracy: 0.8794, F1: 0.7651
Best model saved!
Epoch 3/10
Train Loss: 0.1958, Val Loss: 0.5625, Val Accuracy: 0.8636, F1: 0.7459
Epoch 4/10
Train Loss: 0.1260, Val Loss: 0.8387, Val Accuracy: 0.8718, F1: 0.7586
Epoch 5/10
Train Loss: 0.0738, Val Loss: 0.9818, Val Accuracy: 0.8699, F1: 0.7488
Epoch 6/10
Train Loss: 0.0562, Val Loss: 1.0552, Val Accuracy: 0.8706, F1: 0.7545
Epoch 7/10
Train Loss: 0.0459, Val Loss: 1.0912, Val Accuracy: 0.8674, F1: 0.7470
Early stopping triggered!


In [55]:
state_dict = torch.load("best_model.pth", weights_only=True)  # Explicitly set weights_only=True
model.load_state_dict(state_dict)
print("Best model loaded.")

Best model loaded.


In [56]:
all_preds, all_labels = [], []
with torch.no_grad():
  for texts, labels in test_dataloader:
    output = model(texts).squeeze(1)
    all_preds += torch.round(torch.sigmoid(output)).cpu().tolist()
    all_labels += labels.cpu().tolist()
accuracy = accuracy_score(all_labels,all_preds)
f1 = f1_score(all_labels,all_preds)
print(f"Test Accuracy: {accuracy:.4f}, F1: {f1:.4f}")

Test Accuracy: 0.9003, F1: 0.8115
